In [1]:
###Pythons SA

In [2]:
conda install --yes nltk

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - nltk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py39hf3d152e_0         144 KB  conda-forge
    click-8.0.3                |   py39hf3d152e_0         146 KB  conda-forge
    joblib-1.1.0               |     pyhd8ed1ab_0         210 KB  conda-forge
    nltk-3.6.5                 |     pyhd8ed1ab_0         1.1 MB  conda-forge
    openssl-1.1.1l             |       h7f98852_0         2.1 MB  conda-forge
    regex-2021.10.8            |   py39h3811e60_0         380 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

  click    

In [1]:
import nltk
from nltk import sentiment
from nltk.sentiment.util import *
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib as mpl
import matplotlib.pyplot as plt

from importlib import reload
import saffine
reload(saffine)
import numpy as np
import os
import re
import argparse
import saffine.multi_detrending as md
import saffine.detrending_method as dm
from metadata_parser import work_vals
from scipy.stats import norm

import glob
import time

Matplotlib is building the font cache; this may take a moment.


In [8]:
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ucloud/nltk_data...
[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
sid = SentimentIntensityAnalyzer() #instantiates nltk's sentiment analyzer

In [10]:
### short function to produce sentimental arcs

def sentimarc(text, untokenized=True):
    if untokenized:
        sentences = nltk.sent_tokenize(text)
        print(len(sentences)," sentences")
    else: sentences = text
    sentimental_arc=[]
    for sentence in sentences:
        compound_polarity = sid.polarity_scores(sentence)['compound']
        sentimental_arc.append(compound_polarity)
    return sentimental_arc

In [11]:
## Some functions Thelma's notebooks, but "figures" has an added hack to avoid an error 
# specific arc lengths used to throw. Also it returns the overall Hurst value

def integrate(x):
    return np.mat(np.cumsum(x) - np.mean(x))

def normalize(ts, scl01 = False):
    ts01 = (ts - np.min(ts)) / (np.max(ts) - np.min(ts))
    ts11 = 2 * ts01 -1
    if scl01:
        return ts01
    else:
        return ts11
    
def optimal_bin(y):
    """ optimal number of bins for histogram
    src: https://academic.oup.com/biomet/article-abstract/66/3/605/232642
    """
    R = max(y)-min(y)
    n = len(y)
    sigma = np.std(y)
    return int(round((R * (n**(1./3.))) / (3.49 * sigma)))

# functions to produce figures
def figures(story_arc,sentimethod, workname, workid, neutral_score=0):
    y = integrate(story_arc)
    uneven = y.shape[1]%2
    if uneven:
        y = y[0,:-1]

    # afa
    #n = 500
    step_size = 1
    q = 3
    order = 1
    xy = md.multi_detrending(y, step_size, q, order)
    ## slope
    x = np.squeeze(np.asarray(xy[0]))
    y = np.squeeze(np.asarray(xy[1]))

    p = np.poly1d(np.polyfit(x, y, order))
    xp = np.linspace(0, len(x), len(x))

    #fig, ax = plt.subplots(2,1)

    plt.figure(figsize=(12,20))
    plt.subplot(311)
    X = np.mat([float(x) for x in story_arc])
    plt.plot(X.T,'-k', label = 'story arc')
    n = len(story_arc)
    w = int(4 * np.floor(n/20) + 1)

    # format
    for i in range(2,5):
        try:
            _, trend_ww_1 = dm.detrending_method(X, w, i)
            plt.plot(normalize(trend_ww_1).T, label = "$m = {}$".format(str(i)))
        except:
            print("error")
            X = np.mat([float(x) for x in story_arc+[neutral_score]])
            plt.plot(X.T,'-k', label = 'story arc')
            n = len(story_arc)
            w = int(4 * np.floor(n/20) + 1)
            pass
           
    plt.title("$%s~Story~Arc~{}$".format(str(sentimethod)) % (workname))
    plt.legend()
    plt.xlabel('$t$')
    plt.ylabel('$F(t)$')

    plt.subplot(312)
     # parameters
        
    for i in range(2,5):
        _, trend_ww_1 = dm.detrending_method(X, w, i)
        plt.plot(normalize(trend_ww_1).T, label = "$m = {}$".format(str(i)))

    #plt.title("$Passing~Story~Arc_{Syuzhet}$")
    plt.legend()
    plt.xlabel('$t$')
    plt.ylabel('$F(t)_{-1:1}$')
    
    plt.subplot(325)
    M = np.mean(story_arc)
    SD =  np.std(story_arc)
    n, bins, _ = plt.hist(story_arc, optimal_bin(story_arc) ,density = True , facecolor = 'gray', edgecolor = 'w')
    
    Y = norm.pdf(bins, M, SD)
    plt.plot(bins, Y, 'k-', linewidth=1.5)
    plt.ylabel('$Sentiment~score$')
    plt.ylabel('$Density$')

    
    plt.subplot(326)
    plt.plot(xp, p(xp), 'k-', linewidth = 2,zorder = 0)
    plt.scatter(x, y, c = 'r', s = 50, zorder = 1)
    plt.title('$H = {}$'.format(round(np.polyfit(x, y, 1)[0],2)))
    plt.xlabel('$Log(w)$')
    plt.ylabel('$LogF(w)$')
    
    plt.tight_layout()
    workname = re.sub("\W+"," ", workname.lower())
    workname = re.sub(" +", "_", workname)
    workname = workname +"_{}".format(sentimethod)
    workid = workid +"_{}".format(sentimethod)
    ####plt.savefig(os.path.join("fig", workid))
    plt.savefig(workname+".png")
    
    plt.close()
    
    H = round(np.polyfit(x, y, 1)[0],2)
    return H

In [12]:
##loop on the texts

path_to_texts = "andersen_subset/*.txt"

texts =[]
for name in glob.glob(path_to_texts): 
    tale = open(name).read()
    title = name.split("/")[-1].split(".")[0]
    print(title)
    texts.append((title, tale))

The Ugly Duckling
The Princess and the Pea
The Little Match-seller
The Wild Swans
The Swineherd
Little Tiny or Thumbelina
The Fir Tree
The Snow Man
The Brave Tin Soldier
The Little Mermaid
The Emperor's New Suit
Jack the Dullard


In [13]:
%%time

## Sentence based sentimental arcs with nltk's Vader scores

arcs = []
for text in texts: 
    title, tale = text
    print(title)
    arc = sentimarc(tale)
    arcs.append((title,arc))
    Hurst_value = figures(arc, "nltk_sentences", title, "0") #we would prob. use a way to not save/plot everytime

The Ugly Duckling
189  sentences
The Princess and the Pea
25  sentences
error
The Little Match-seller
46  sentences
The Wild Swans
287  sentences
The Swineherd
95  sentences
Little Tiny or Thumbelina
192  sentences
The Fir Tree
185  sentences
The Snow Man
124  sentences
The Brave Tin Soldier
81  sentences
error
The Little Mermaid
332  sentences
The Emperor's New Suit
95  sentences
Jack the Dullard
122  sentences
CPU times: user 11.3 s, sys: 23.2 s, total: 34.5 s
Wall time: 8.41 s


In [14]:
[a[0] for a in arcs]

['The Ugly Duckling',
 'The Princess and the Pea',
 'The Little Match-seller',
 'The Wild Swans',
 'The Swineherd',
 'Little Tiny or Thumbelina',
 'The Fir Tree',
 'The Snow Man',
 'The Brave Tin Soldier',
 'The Little Mermaid',
 "The Emperor's New Suit",
 'Jack the Dullard']

In [15]:
%%time

# Sentence based sentimental arcs with nltk's Vader scores

Hs={} #Dictionary to store Hurst values


out = open("sentence_sentiments.txt","w") #output file 

for text_t in texts:
    title, text = text_t
    out.write(title+"\n")
    print(title)
    
    sentences = nltk.sent_tokenize(text)
        
    s_arc = [sid.polarity_scores(sentence)["compound"] for sentence in sentences]
    
    #word_arc = [sid.polarity_scores(w)["compound"] for w in words]
    
    print(len(sentences),len(s_arc))
    print("Average score: ", np.mean(s_arc))
    print("First score: ",s_arc[0],"\nLast score: ",s_arc[-1])
    if len(s_arc)>20: H=figures(s_arc,"ntlk_V",title+"_words","w")
    print("H: ",str(H))
    out.write(str(s_arc))
    out.write("\n"+str(H))
    out.write("\n\n")
    
    Hs[title]=[H]
    
    print()
    
out.close()

The Ugly Duckling
189 189
Average score:  0.002982539682539689
First score:  0.5672 
Last score:  0.296
H:  0.61

The Princess and the Pea
25 25
Average score:  -0.029635999999999996
First score:  0.0 
Last score:  0.4215
error
H:  0.63

The Little Match-seller
46 46
Average score:  0.13603695652173914
First score:  -0.6369 
Last score:  -0.0387
H:  0.69

The Wild Swans
287 287
Average score:  0.07832020905923344
First score:  0.0 
Last score:  -0.296
H:  0.57

The Swineherd
95 95
Average score:  0.14164105263157897
First score:  -0.2617 
Last score:  0.5106
H:  0.63

Little Tiny or Thumbelina
192 192
Average score:  0.13950156249999998
First score:  -0.438 
Last score:  0.0
H:  0.6

The Fir Tree
185 185
Average score:  0.15637027027027026
First score:  0.8361 
Last score:  0.0
H:  0.57

The Snow Man
124 124
Average score:  0.09159193548387098
First score:  0.6948 
Last score:  0.0
H:  0.6

The Brave Tin Soldier
81 81
Average score:  0.05664938271604938
First score:  0.6289 
Last score

In [16]:
%%time

# Word-by-word sentimental arcs with nltk's Vader scores

Hs={} #Dictionary to store Hurst values


out = open("word_filtered_sentiments_Vader.txt","w") #output file 

for text_t in texts:
    title, text = text_t
    out.write(title+"\n")
    print(title)
    words = nltk.wordpunct_tokenize(text)
    
    #Here I only keep words that are in sid's lexicon. So no neutral words
    words_in_sid_lexicon = [w for w in words if w in sid.lexicon.keys()] 
    
    word_arc = [sid.polarity_scores(w)["compound"] for w in words_in_sid_lexicon]
    
    #word_arc = [sid.polarity_scores(w)["compound"] for w in words]
    
    print(len(words),len(word_arc))
    print("Average score: ", np.mean(word_arc))
    print("First score: ",word_arc[0],"\nLast score: ",word_arc[-1])
    if len(word_arc)>20: H=figures(word_arc,"ntlk_V",title+"_words","w")
    print("H: ",str(H))
    out.write(str(word_arc))
    out.write("\n"+str(H))
    out.write("\n\n")
    
    Hs[title]=[H]
    
    print()
    
out.close()

The Ugly Duckling
4507 276
Average score:  0.028526449275362315
First score:  0.5859 
Last score:  -0.5106
H:  0.7

The Princess and the Pea
446 14
Average score:  -0.12212857142857146
First score:  -0.3612 
Last score:  0.4215
H:  0.7

The Little Match-seller
1170 65
Average score:  0.12561692307692307
First score:  -0.5574 
Last score:  0.5423
H:  0.76

The Wild Swans
7364 361
Average score:  0.08266814404432132
First score:  0.34 
Last score:  -0.296
error
H:  0.65

The Swineherd
1862 106
Average score:  0.1740160377358491
First score:  -0.4767 
Last score:  0.5106
H:  0.59

Little Tiny or Thumbelina
5143 292
Average score:  0.13879931506849316
First score:  0.4019 
Last score:  0.2263
H:  0.64

The Fir Tree
3964 203
Average score:  0.19528128078817733
First score:  0.2263 
Last score:  0.6369
H:  0.66

The Snow Man
2457 101
Average score:  0.2191930693069307
First score:  0.5719 
Last score:  -0.4215
error
H:  0.73

The Brave Tin Soldier
2032 78
Average score:  0.08982692307692308


In [17]:
Hs.values()

dict_values([[0.7], [0.7], [0.76], [0.65], [0.59], [0.64], [0.66], [0.73], [0.68], [0.71], [0.63], [0.53]])

In [18]:
#### To compare VAD scores, I create a VAD dictionary

#"/Users/au701203/Downloads/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt"

path_to_the_lexicon = "NRC-VAD-Lexicon.txt"

VAD = open(path_to_the_lexicon).read()
lines = VAD.split("\n")
tabs = [l.split("\t") for l in lines]
tabs = [t for t in tabs if len(t)==4]
Vadscores = {}
for tab in tabs:
    Vadscores[tab[0]]=float(tab[1])

In [19]:
Vadscores["glory"]  #Vad scores' minimum is 0

0.948

In [20]:
### VAD arcs

In [32]:
%%time

# Vader word-by-word sentiment arcs using VAD scores

out = open("word_filtered_sentiments_VAD.txt","w")

for text_t in texts:
    title, text = text_t
    print(title)
    words = nltk.wordpunct_tokenize(text)
    
    #Using VAD's lexicon
    words_in_vad_lexicon = [w for w in words if w in Vadscores.keys()]
    word_arc = [Vadscores[w] for w in words_in_vad_lexicon]
    
    print(len(words),len(word_arc))
    
    print(word_arc[:10])
    print("Average happiness: ", np.mean(word_arc))
    print("First score: ",word_arc[0],"\nLast score: ",word_arc[-1])
    if len(word_arc)>20: H=figures(word_arc,"VAD",title+"_words","w")
    print("H: ",str(H))
    out.write(str(word_arc))
    out.write("\n"+str(H))
    out.write("\n\n")
    
    Hs[title].append(H)
    
    print()
    
out.close()

The Ugly Duckling
4507 1174
[0.917, 0.812, 0.755, 0.698, 0.802, 0.594, 0.63, 0.875, 0.708, 0.541]
Average happiness:  0.5891669505962521
First score:  0.917 
Last score:  0.602
H:  0.63

The Princess and the Pea
446 108
[0.604, 0.888, 0.896, 0.885, 0.493, 0.757, 0.67, 0.719, 0.885, 0.622]
Average happiness:  0.6107777777777779
First score:  0.604 
Last score:  0.653
H:  0.66

The Little Match-seller
1170 325
[0.061, 0.398, 0.198, 0.458, 0.542, 0.667, 0.202, 0.398, 0.102, 0.156]
Average happiness:  0.5582738461538462
First score:  0.061 
Last score:  0.719
H:  0.57

The Wild Swans
7364 1898
[0.582, 0.76, 0.663, 0.677, 0.449, 0.542, 0.908, 0.562, 0.449, 0.765]
Average happiness:  0.6014199157007376
First score:  0.582 
Last score:  0.677
H:  0.63

The Swineherd
1862 443
[0.604, 0.156, 0.888, 0.714, 0.542, 0.704, 0.76, 0.896, 0.896, 0.493]
Average happiness:  0.6197968397291196
First score:  0.604 
Last score:  0.385
H:  0.53

Little Tiny or Thumbelina
5143 1392
[0.757, 0.385, 0.912, 0.76

In [33]:
Hs

{'The Ugly Duckling': [0.7, 0.63],
 'The Princess and the Pea': [0.7, 0.66],
 'The Little Match-seller': [0.76, 0.57],
 'The Wild Swans': [0.65, 0.63],
 'The Swineherd': [0.59, 0.53],
 'Little Tiny or Thumbelina': [0.64, 0.62],
 'The Fir Tree': [0.66, 0.61],
 'The Snow Man': [0.73, 0.57],
 'The Brave Tin Soldier': [0.68, 0.63],
 'The Little Mermaid': [0.71, 0.66],
 "The Emperor's New Suit": [0.63, 0.54],
 'Jack the Dullard': [0.53, 0.51]}

In [3]:
#average of Hs
ranks = []
for k in Hs.keys():
    hs = Hs[k]
    #print(k+"\t",round(np.mean(hs),3), round(np.std(hs),3))
    ranks.append((np.mean(hs),k))
    
ranks.sort()
print(ranks)

NameError: name 'Hs' is not defined

In [ ]:
## My clustering (all in a notebook for now, to go smoother)

In [ ]:
from tslearn.utils import to_time_series_dataset
from tslearn.preprocessing import TimeSeriesResampler
from matplotlib import pyplot as plt

In [ ]:
# %%
import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    
def plot_hierarchy_2D(X, labels=[], title='Hierarchical Clustering Dendrogram', node_col='newspaper_event'):
    #df.index = df[node_col]
    #X = np.array(df[['X', 'Y']])

    # setting distance_threshold=0 ensures we compute the full tree.
    
    new_shape = []
    for el in X:
        new_shape.append([float(e) for e in el])

    new_shape = np.array(new_shape)
    
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

    model = model.fit(new_shape)
    print(model.n_connected_components_, model.n_clusters_, model.labels_)
    fig = plt.figure(figsize=(20, 10))
    plt.title(title)
    # plot the top three levels of the dendrogram
    plot_dendrogram(model, truncate_mode='level', p=7, labels=labels)# df.index)
    plt.xticks(rotation=90)
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    
    return fig 

In [ ]:
labels, warcs, warcs_vader, warcs_vad = [],[],[],[]
for key in word_arcs.keys():
    labels.append(key)
    labels.append(key+"_VAD")
    warcs+=word_arcs[key]
    vader = word_arcs[key][0]
    vad = word_arcs[key][1]
    #vad_ = [e-.1 for e in vad]
    warcs_vad.append(vad)
    warcs_vader.append(vader)
    #warcs_vad.append(vad_)

In [ ]:
n,m = 0,12
series = warcs_vad[n:m]
leng = min([len(e) for e in series])
print(leng)
serialized_warcs = to_time_series_dataset(series)
resampled_warcs = TimeSeriesResampler(sz=3000).fit_transform(serialized_warcs) 
labels_ = clean_ls[n:m] #
print(labels_)
print([len(s) for s in series])
plot_hierarchy_2D(resampled_warcs, labels=labels_, title="vad3000")

In [3]:
import sys
!{sys.executable} -m pip install flair

     |████████████████████████████████| 319 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 881.9 MB 2.1 kB/s s eta 0:00:01     |██████████████▎                 | 393.5 MB 109.4 MB/s eta 0:00:05 MB/s eta 0:00:02��████████████████        | 663.4 MB 129.1 MB/s eta 0:00:02     |████████████████████████▉       | 684.9 MB 119.0 MB/s eta 0:00:02[K     |██████████████████████████▊     | 736.1 MB 128.3 MB/s eta 0:00:02
     |████████████████████████████████| 24.7 MB 67.2 MB/s eta 0:00:01
     |████████████████████████████████| 19.7 MB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 23.4 MB 33.8 MB/s eta 0:00:01                     | 1.2 MB 33.8 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 565 kB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 64.4 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 44.8 MB/s eta 0:00:01
     |████████████████████████████

In [4]:
import flair

In [5]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-10-21 17:18:47,215 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpsc41ilqg


100%|██████████| 265512723/265512723 [00:06<00:00, 39566059.76B/s]

2021-10-21 17:18:54,036 copying /tmp/tmpsc41ilqg to cache at /home/ucloud/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-10-21 17:18:54,257 removing temp file /tmp/tmpsc41ilqg
2021-10-21 17:18:54,290 loading file /home/ucloud/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
sentence = "I am so frustrated with you right now".split()
s = flair.data.Sentence(sentence)

In [10]:
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment

[NEGATIVE (0.9987)]

In [11]:
import sys
!{sys.executable} -m pip install tensorflow

     |████████████████████████████████| 458.4 MB 9.1 kB/s eta 0:00:0103     |████████████████████████████    | 400.4 MB 3.0 MB/s eta 0:00:20
     |████████████████████████████████| 42 kB 79 kB/s s eta 0:00:01
     |████████████████████████████████| 1.3 MB 62.3 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 56.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 49.7 MB/s eta 0:00:01     |█████████████████████████████   | 13.5 MB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 72.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 51.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 203 kB/s  eta 0:00:01
     |████████████████████████████████| 4.9 MB 43.3 MB/s eta 0:00:01
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30702 sha256=26a02bbc88651466264739fb566cd45f3c99cbcfd502bb12dd0d935a2a5f4cd6
  Stored in directory: /h

In [12]:
import tensorflow

2021-10-21 17:26:12.638216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 17:26:12.638309: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
